In [1]:
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
import argparse
import os 
import os.path as osp
from darknet import Darknet
import pickle as pkl
import pandas as pd
import random

In [2]:
num_classes = 80    #For COCO
classes = load_classes("data/coco.names")
 
images = "imgs"
det="det"
batch_size = 1
confidence = 0.5
nms_thesh = 0.4
start = 0
CUDA = torch.cuda.is_available()

In [3]:
print (classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [4]:
#Set up the neural network
print("Loading network.....")
model = Darknet( "cfg/yolov3.cfg")

Loading network.....


In [5]:
print(model)

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1, inplace)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1, inplace)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_3): Batch

In [6]:
model.load_weights("yolov3.weights")
print("Network successfully loaded")

Network successfully loaded


In [7]:
model.net_info["height"] = 320

In [8]:
inp_dim = int(model.net_info["height"])

In [9]:
print(inp_dim)

320


In [10]:
assert inp_dim % 32 == 0 
assert inp_dim > 32

In [11]:
#Set the model in evaluation mode
model.eval()

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1, inplace)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1, inplace)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_3): Batch

In [12]:
read_dir = time.time()

In [13]:
print(read_dir)

1540448268.0722437


In [14]:
images="imgs"
imlist = [osp.join(osp.realpath('.'), images, img) for img in os.listdir(images)]

In [15]:
print(imlist)

['D:\\yolov3\\imgs\\dog.jpg', 'D:\\yolov3\\imgs\\eagle.jpg', 'D:\\yolov3\\imgs\\giraffe.jpg', 'D:\\yolov3\\imgs\\herd_of_horses.jpg', 'D:\\yolov3\\imgs\\img1.jpg', 'D:\\yolov3\\imgs\\img2.jpg', 'D:\\yolov3\\imgs\\img3.jpg', 'D:\\yolov3\\imgs\\img4.jpg', 'D:\\yolov3\\imgs\\messi.jpg', 'D:\\yolov3\\imgs\\person.jpg', 'D:\\yolov3\\imgs\\scream.jpg']


In [16]:
load_batch = time.time()
loaded_ims = [cv2.imread(x) for x in imlist]

In [17]:
print(loaded_ims)

[array([[[ 50,  58,  57],
        [ 51,  59,  58],
        [ 53,  61,  60],
        ...,
        [ 47,  89, 142],
        [ 41,  50,  88],
        [ 47,  71,  63]],

       [[ 51,  59,  58],
        [ 51,  59,  58],
        [ 52,  60,  59],
        ...,
        [ 37,  74, 124],
        [ 41,  50,  83],
        [ 46,  70,  58]],

       [[ 51,  59,  58],
        [ 51,  59,  58],
        [ 52,  60,  59],
        ...,
        [ 25,  54,  98],
        [ 48,  54,  77],
        [ 43,  62,  45]],

       ...,

       [[179, 168, 160],
        [179, 168, 160],
        [182, 171, 163],
        ...,
        [ 64,  64,  80],
        [ 36,  39,  53],
        [ 48,  53,  62]],

       [[180, 169, 161],
        [179, 168, 160],
        [175, 164, 156],
        ...,
        [ 61,  61,  79],
        [ 37,  40,  54],
        [ 52,  56,  67]],

       [[176, 165, 157],
        [178, 167, 159],
        [172, 161, 153],
        ...,
        [ 62,  62,  80],
        [ 33,  36,  50],
        [ 35,  39,  50]

In [18]:
#PyTorch Variables for images
im_batches = list(map(prep_image, loaded_ims, [inp_dim for x in range(len(imlist))]))

In [19]:
print(im_batches)

[tensor([[[[0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          ...,
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020]],

         [[0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          ...,
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020]],

         [[0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 

In [20]:
#List containing dimensions of original images
im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)

In [21]:
 
leftover = 0
if (len(im_dim_list) % batch_size):
   leftover = 1

In [22]:
if batch_size != 1:
   num_batches = len(imlist) // batch_size + leftover            
   im_batches = [torch.cat((im_batches[i*batch_size : min((i +  1)*batch_size,
                       len(im_batches))]))  for i in range(num_batches)]  

In [23]:
write = 0
start_det_loop = time.time()

In [24]:
 
for i, batch in enumerate(im_batches):
    #load the image 
    start = time.time()
    if CUDA:
        batch = batch.cuda()

    prediction = model(Variable(batch, volatile = True), CUDA)

    prediction = write_results(prediction, confidence, num_classes, nms_conf = nms_thesh)

    end = time.time()

    if type(prediction) == int:

        for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
            im_id = i*batch_size + im_num
            print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
            print("{0:20s} {1:s}".format("Objects Detected:", ""))
            print("----------------------------------------------------------")
        continue

    prediction[:,0] += i*batch_size    #transform the atribute from index in batch to index in imlist 

    if not write:                      #If we have't initialised output
        output = prediction  
        write = 1
    else:
        output = torch.cat((output,prediction))
    print("output:"+str(output))
    for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
        im_id = i*batch_size + im_num
        objs = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]
        print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
        print("{0:20s} {1:s}".format("Objects Detected:", " ".join(objs)))
        print("----------------------------------------------------------")

    if CUDA:
        torch.cuda.synchronize() 

c:\users\faizan\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
c:\users\faizan\appdata\local\programs\python\python35\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


output:tensor([[  0.0000,  48.0398,  84.7070, 235.2344, 222.3178,   0.9981,   0.9996,
           1.0000],
        [  0.0000, 199.4577,  76.3405, 288.2465, 110.9368,   0.9995,   0.7089,
           7.0000],
        [  0.0000,  50.8519, 134.0979, 132.0082, 263.4641,   0.9999,   0.9958,
          16.0000]])
D:\yolov3\imgs\dog.jpg predicted in  3.531 seconds
Objects Detected:    bicycle truck dog
----------------------------------------------------------
output:tensor([[  0.0000,  48.0398,  84.7070, 235.2344, 222.3178,   0.9981,   0.9996,
           1.0000],
        [  0.0000, 199.4577,  76.3405, 288.2465, 110.9368,   0.9995,   0.7089,
           7.0000],
        [  0.0000,  50.8519, 134.0979, 132.0082, 263.4641,   0.9999,   0.9958,
          16.0000],
        [  1.0000,  47.8960,  86.4021, 255.4880, 243.5085,   0.9998,   0.9987,
          14.0000]])
D:\yolov3\imgs\eagle.jpg predicted in  2.500 seconds
Objects Detected:    bird
----------------------------------------------------------
outp

output:tensor([[  0.0000,  48.0398,  84.7070, 235.2344, 222.3178,   0.9981,   0.9996,
           1.0000],
        [  0.0000, 199.4577,  76.3405, 288.2465, 110.9368,   0.9995,   0.7089,
           7.0000],
        [  0.0000,  50.8519, 134.0979, 132.0082, 263.4641,   0.9999,   0.9958,
          16.0000],
        [  1.0000,  47.8960,  86.4021, 255.4880, 243.5085,   0.9998,   0.9987,
          14.0000],
        [  2.0000, 158.6947, 123.4248, 272.6176, 288.8128,   0.9984,   0.9645,
          22.0000],
        [  2.0000,  92.2093,  18.9892, 287.7239, 278.5399,   0.9981,   0.9994,
          23.0000],
        [  3.0000, 181.6146, 140.6180, 250.9958, 195.8427,   0.9984,   0.6691,
          17.0000],
        [  3.0000,   0.3138, 132.9995, 141.4387, 228.3570,   0.9965,   0.9958,
          17.0000],
        [  3.0000,  96.2467, 127.6147, 190.5748, 204.2887,   0.9189,   0.9733,
          17.0000],
        [  4.0000,  36.7090,   8.8220, 273.5977, 325.3773,   0.9891,   0.9923,
           0.0000],
   

In [25]:
im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())

scaling_factor = torch.min(inp_dim/im_dim_list,1)[0].view(-1,1)


output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2

In [26]:
output[:,1:5] /= scaling_factor

In [27]:
for i in range(output.shape[0]):
    output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim_list[i,0])
    output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim_list[i,1])

In [28]:
class_load = time.time()
colors = pkl.load(open("pallete", "rb"))

In [29]:
draw = time.time()

def write(x, results):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results[int(x[0])]
    cls = int(x[-1])
    color = random.choice(colors)
    label = "{0}".format(classes[cls])
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img

In [30]:
list(map(lambda x: write(x, loaded_ims), output))

[array([[[ 50,  58,  57],
         [ 51,  59,  58],
         [ 53,  61,  60],
         ...,
         [ 47,  89, 142],
         [ 41,  50,  88],
         [ 47,  71,  63]],
 
        [[ 51,  59,  58],
         [ 51,  59,  58],
         [ 52,  60,  59],
         ...,
         [ 37,  74, 124],
         [ 41,  50,  83],
         [ 46,  70,  58]],
 
        [[ 51,  59,  58],
         [ 51,  59,  58],
         [ 52,  60,  59],
         ...,
         [ 25,  54,  98],
         [ 48,  54,  77],
         [ 43,  62,  45]],
 
        ...,
 
        [[179, 168, 160],
         [179, 168, 160],
         [182, 171, 163],
         ...,
         [ 64,  64,  80],
         [ 36,  39,  53],
         [ 48,  53,  62]],
 
        [[180, 169, 161],
         [179, 168, 160],
         [175, 164, 156],
         ...,
         [ 61,  61,  79],
         [ 37,  40,  54],
         [ 52,  56,  67]],
 
        [[176, 165, 157],
         [178, 167, 159],
         [172, 161, 153],
         ...,
         [ 62,  62,  80],
  

In [31]:
 print(loaded_ims)

[array([[[ 50,  58,  57],
        [ 51,  59,  58],
        [ 53,  61,  60],
        ...,
        [ 47,  89, 142],
        [ 41,  50,  88],
        [ 47,  71,  63]],

       [[ 51,  59,  58],
        [ 51,  59,  58],
        [ 52,  60,  59],
        ...,
        [ 37,  74, 124],
        [ 41,  50,  83],
        [ 46,  70,  58]],

       [[ 51,  59,  58],
        [ 51,  59,  58],
        [ 52,  60,  59],
        ...,
        [ 25,  54,  98],
        [ 48,  54,  77],
        [ 43,  62,  45]],

       ...,

       [[179, 168, 160],
        [179, 168, 160],
        [182, 171, 163],
        ...,
        [ 64,  64,  80],
        [ 36,  39,  53],
        [ 48,  53,  62]],

       [[180, 169, 161],
        [179, 168, 160],
        [175, 164, 156],
        ...,
        [ 61,  61,  79],
        [ 37,  40,  54],
        [ 52,  56,  67]],

       [[176, 165, 157],
        [178, 167, 159],
        [172, 161, 153],
        ...,
        [ 62,  62,  80],
        [ 33,  36,  50],
        [ 35,  39,  50]

In [32]:
print(loaded_ims[0])

[[[ 50  58  57]
  [ 51  59  58]
  [ 53  61  60]
  ...
  [ 47  89 142]
  [ 41  50  88]
  [ 47  71  63]]

 [[ 51  59  58]
  [ 51  59  58]
  [ 52  60  59]
  ...
  [ 37  74 124]
  [ 41  50  83]
  [ 46  70  58]]

 [[ 51  59  58]
  [ 51  59  58]
  [ 52  60  59]
  ...
  [ 25  54  98]
  [ 48  54  77]
  [ 43  62  45]]

 ...

 [[179 168 160]
  [179 168 160]
  [182 171 163]
  ...
  [ 64  64  80]
  [ 36  39  53]
  [ 48  53  62]]

 [[180 169 161]
  [179 168 160]
  [175 164 156]
  ...
  [ 61  61  79]
  [ 37  40  54]
  [ 52  56  67]]

 [[176 165 157]
  [178 167 159]
  [172 161 153]
  ...
  [ 62  62  80]
  [ 33  36  50]
  [ 35  39  50]]]


In [33]:
det_names = pd.Series(imlist).apply(lambda x:  )

SyntaxError: invalid syntax (<ipython-input-33-0af4b0fadd7c>, line 1)

In [ ]:
print(det_names)

In [ ]:
print(imlist)